
## Detectron2 is Facebooks new vision library 




In [1]:
!pip install --user pyyaml==5.1
!pip show pyyaml  


You should consider upgrading via the 'C:\Users\HP\anaconda3\envs\FaceRec\python.exe -m pip install --upgrade pip' command.


Name: PyYAML
Version: 5.1
Summary: YAML parser and emitter for Python
Home-page: https://github.com/yaml/pyyaml
Author: Kirill Simonov
Author-email: xi@resolvent.net
License: MIT
Location: c:\users\hp\appdata\roaming\python\python36\site-packages
Requires: 
Required-by: Keras, yacs, omegaconf, fvcore, dask, albumentations


In [2]:
import tensorflow as tf
tf.__version__

'2.4.0'

In [ ]:
#!nvidia-smi
!nvcc --version #10.1
!python --version # 3.7.6

In [ ]:
# CPU Only
#conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cpuonly -c pytorch

# CUDA 10.1
#!conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cudatoolkit=10.1 -c pytorch

#!pip3 install torch==1.8.0 torchvision==0.9.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


In [3]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.8.0 False


## Other Requirements

1. fvcore: pip install git+https://github.com/facebookresearch/fvcore

2. pycocotools: pip install cython; pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI



In [ ]:
!git clone https://github.com/philferriere/cocoapi.git

In [4]:
%cd cocoapi/PythonAPI
!python setup.py build_ext install

[WinError 3] The system cannot find the path specified: 'cocoapi/PythonAPI'
C:\Users\HP\Mini Project\Detectron2-Tutorial


python: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

In [5]:
%cd ../../detectron2

[WinError 2] The system cannot find the file specified: '../../detectron2'
C:\Users\HP\Mini Project\Detectron2-Tutorial


In [6]:
!python setup.py build develop

python: can't open file 'setup.py': [Errno 2] No such file or directory


Inference with pre-trained model

Using a pre-trained model is super easy in Detectron2. You only need to load in a config and some weights and then create a DefaultPredictor. After that you can simply make predictions and display them using Detectron's Visualizer utility.

In [23]:
# import some common detectron2 utilities
from detectron2 import *
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
import matplotlib.pyplot as plt
path="../dataset/Test004/"
im = cv2.imread("y.tif")
#cv2.imwrite("x.jpg",img,[int(cv2.IMWRITE_JPEG_QUALITY), 200])
#im = cv2.imread(path+"x.jpg")

In [24]:
#Show the image with matplotlib
#from IPython.display import Image
#Image('../download.jpg')


plt.figure(figsize=(15,7.5))
plt.imshow(im[0, ::-1]) #bgr to rgb

In [25]:
# Create config
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"


In [26]:
# Create predictor

# If getting AssertionError: Torch not compiled with CUDA enabled detectron2. Then do the below given line
cfg.MODEL.DEVICE = "cpu"

predictor = DefaultPredictor(cfg)

In [27]:
outputs = predictor(im)

In [28]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

tensor([ 0,  0,  0,  0,  0,  0,  7,  0,  0,  0, 24,  0,  0, 24, 36])
Boxes(tensor([[ 51.8325, 116.8473,  71.9855, 147.9277],
        [ 29.1780, 119.0322,  41.0629, 150.8572],
        [234.0138, 129.0095, 255.1849, 162.7054],
        [148.2239, 114.6921, 166.0296, 149.9127],
        [101.6209, 107.8644, 111.9267, 138.4512],
        [339.2444,  95.0136, 351.4698, 123.8319],
        [244.6751, 142.3254, 325.8720, 191.5563],
        [301.5641, 100.5347, 312.3703, 129.8334],
        [290.5341, 106.8128, 301.5305, 134.2334],
        [194.1723, 212.8246, 210.5061, 240.0000],
        [194.4763, 219.5479, 205.4425, 234.5256],
        [150.5106, 104.5667, 164.1210, 135.2664],
        [227.0921,  94.7503, 239.8811, 118.4814],
        [246.9312, 136.5886, 253.4363, 146.9085],
        [339.8276, 118.8889, 350.1174, 123.5877]]))


In [29]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))


def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(25,7.5)), plt.imshow(im), plt.axis('off');
plt.figure(figsize=(10,5))
#print(out.get_image()[:, :, ::-1])
cv2.imwrite("y1.jpg",out.get_image()[:, :, ::-1])

True

## keypoint detection  : detecting people and localizing their keypoints.

In [30]:
from detectron2 import model_zoo

# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config
cfg.MODEL.DEVICE = "cpu"

cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

im = cv2.imread("y.tif")

def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(25,7.5)), plt.imshow(im), plt.axis('off');

predictor = DefaultPredictor(cfg)
outputs = predictor(im)

v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))


def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(25,7.5)), plt.imshow(im), plt.axis('off');
plt.figure(figsize=(10,5))
#print(out.get_image()[:, :, ::-1])
cv2.imwrite("y2.jpg",out.get_image()[:, :, ::-1])

True

## Panoptic segmentation model

In [31]:

cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"

cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")

predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2.imwrite("y3.jpg",out.get_image()[:, :, ::-1])

True

# Image Segmentation

In [32]:
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"

# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [33]:

v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))


In [34]:
cv2.imwrite("y4.jpg",out.get_image()[:, :, ::-1])

True